In [ ]:
%tensorflow_version 1.x
!pip install tflearn
!pip install tensorboardcolab
!pip install pandas==0.24

TensorFlow 1.x selected.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir("/content/drive/My Drive/intern/PG_stock")

In [ ]:
import json
import time
import sys
import collections
from multiprocessing import Process
from datetime import datetime
from __future__ import division,absolute_import,print_function
import numpy as np
import pandas as pd
import logging
import sqlite3
import tensorflow as tf
import tflearn

#一、必备工具

##1.数据处理

In [ ]:
#填充缺失数据
def panel_fillna(panel, type="bfill"):
    """
    fill nan along the 3rd axis
    :param panel: the panel to be filled
    :param type: bfill or ffill
    """
    frames = {}
    for item in panel.items:
        if type == "both":
            frames[item] = panel.loc[item].fillna(axis=1, method="bfill").fillna(axis=1, method="ffill")
        else:
            frames[item] = panel.loc[item].fillna(axis=1, method=type)
    return pd.Panel(frames)

In [ ]:
#输入起止时间跨度、测试集比例，返回向前推进的时间长度
def get_volume_forward(time_span, portion, portion_reversed):
    volume_forward = 0
    if not portion_reversed:
        volume_forward = time_span*portion
    return volume_forward

##2.config处理

In [ ]:
#设置config参数
def parse_time(time_string):
    return time.mktime(datetime.strptime(time_string, "%Y/%m/%d").timetuple())
def set_missing(config, name, value):
    if name not in config:
        config[name] = value
def fill_layers_default(layers):
    for layer in layers:
        if layer["type"] == "ConvLayer":
            set_missing(layer, "padding", "valid")
            set_missing(layer, "strides", [1, 1])
            set_missing(layer, "activation_function", "relu")
            set_missing(layer, "regularizer", None)
            set_missing(layer, "weight_decay", 0.0)
        elif layer["type"] == "EIIE_Dense":
            set_missing(layer, "activation_function", "relu")
            set_missing(layer, "regularizer", None)
            set_missing(layer, "weight_decay", 0.0)
        elif layer["type"] == "DenseLayer":
            set_missing(layer, "activation_function", "relu")
            set_missing(layer, "regularizer", None)
            set_missing(layer, "weight_decay", 0.0)
        elif layer["type"] == "EIIE_LSTM" or layer["type"] == "EIIE_RNN":
            set_missing(layer, "dropouts", None)
        elif layer["type"] == "EIIE_Output" or\
                layer["type"] == "Output_WithW" or\
                layer["type"] == "EIIE_Output_WithW":
            set_missing(layer, "regularizer", None)
            set_missing(layer, "weight_decay", 0.0)
        elif layer["type"] == "DropOut":
            pass
        else:
            raise ValueError("layer name {} not supported".format(layer["type"]))
def fill_input_default(input_config):
    set_missing(input_config, "save_memory_mode", False)
    set_missing(input_config, "portion_reversed", False)
    set_missing(input_config, "market", "poloniex")
    set_missing(input_config, "norm_method", "absolute")
    set_missing(input_config, "is_permed", False)
    set_missing(input_config, "fake_ratio", 1)
def fill_train_config(train_config):
    set_missing(train_config, "fast_train", True)
    set_missing(train_config, "decay_rate", 1.0)
    set_missing(train_config, "decay_steps", 50000)
def preprocess_config(config):
    set_missing(config, "random_seed", 0)
    set_missing(config, "agent_type", "NNAgent")
    fill_layers_default(config["layers"])
    fill_input_default(config["input"])
    fill_train_config(config["training"])
    return config
def load_config(index=None):
    """
    @:param index: if None, load the default in pgportfolio;
     if a integer, load the config under train_package
    """
    if index:
        with open(os.path.abspath('')+"/train_package/" + str(index) + "/net_config.json") as file:
            config = json.load(file)
    else:
        with open(os.path.abspath('')+"/"+"net_config.json") as file:
            config = json.load(file)
    return preprocess_config(config)

##3.交易过程处理

In [ ]:
#扣除手续费
def calculate_pv_after_commission(w1, w0, commission_rate):
    """
    @:param w1: target portfolio vector, first element is btc
    @:param w0: rebalanced last period portfolio vector, first element is btc
    @:param commission_rate: rate of commission fee, proportional to the transaction cost
    """
    mu0 = 1
    mu1 = 1 - 2*commission_rate + commission_rate ** 2
    while abs(mu1-mu0) > 1e-10:
        mu0 = mu1
        mu1 = (1 - commission_rate * w0[0] -
            (2 * commission_rate - commission_rate ** 2) *
            np.sum(np.maximum(w0[1:] - mu1*w1[1:], 0))) / \
            (1 - commission_rate * w1[0])
    return mu1

##4.生成训练文件夹

In [ ]:
#生成训练文件夹保存模型结果
def add_packages(config, repeat=1):
    train_dir = "train_package"
    package_dir = os.path.abspath('')+'/'+train_dir
    all_subdir = [int(s) for s in os.listdir(package_dir) if os.path.isdir(package_dir+"/"+s)]
    if all_subdir:
        max_dir_num = max(all_subdir)
    else:
        max_dir_num = 0
    indexes = []

    for i in range(repeat):
        max_dir_num += 1
        directory = package_dir+"/"+str(max_dir_num)
        config["random_seed"] = i
        os.makedirs(directory)
        indexes.append(max_dir_num)
        with open(directory + "/" + "net_config.json", 'w') as outfile:
            json.dump(config, outfile, indent=4, sort_keys=True)
    logging.info("create indexes %s" % indexes)
    return indexes

In [ ]:
logging.basicConfig(level=logging.INFO)
add_packages(load_config(), 1)

INFO:root:create indexes [8]


[8]

#二、数据获取

##1.读取股票数据

In [ ]:
#数据存储在df.csv中
conf = load_config()
#特征数量可以是3、4、5
if conf['input']['feature_number']==3:
  feature_list = ["close", "high", "low"]
elif conf['input']['feature_number']==4:
  feature_list = ["open", "close", "high", "low"]
elif conf['input']['feature_number']==5:
  feature_list = ["open", "close", "high", "low", "volume"]
dt = pd.read_csv('dt1.csv',index_col=0)
dt = dt[dt.tdate>int(conf['input']['start_date'][:4])*1e4].reset_index(drop=True)
dt['tdate'] = pd.to_datetime(dt.tdate.astype(str))
date_ind = list(dt.tdate.drop_duplicates())
stock_ind = list(dt.id.drop_duplicates())
data = {}
for feature in feature_list:
  data[feature] = dt.pivot(index='id',columns='tdate',values=feature).iloc[10:21]
  #data[feature] = (data0-data0.mean())/(data0.std())
#将数据转化成面板格式
pn = pd.Panel(data)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
data['close']

tdate,2005-01-04 00:00:00,2005-01-05 00:00:00,2005-01-06 00:00:00,2005-01-07 00:00:00,2005-01-10 00:00:00,2005-01-11 00:00:00,2005-01-12 00:00:00,2005-01-13 00:00:00,2005-01-14 00:00:00,2005-01-17 00:00:00,2005-01-18 00:00:00,2005-01-19 00:00:00,2005-01-20 00:00:00,2005-01-21 00:00:00,2005-01-24 00:00:00,2005-01-25 00:00:00,2005-01-26 00:00:00,2005-01-27 00:00:00,2005-01-28 00:00:00,2005-01-31 00:00:00,2005-02-01 00:00:00,2005-02-02 00:00:00,2005-02-03 00:00:00,2005-02-04 00:00:00,2005-02-16 00:00:00,2005-02-17 00:00:00,2005-02-18 00:00:00,2005-02-21 00:00:00,2005-02-22 00:00:00,2005-02-23 00:00:00,2005-02-24 00:00:00,2005-02-25 00:00:00,2005-02-28 00:00:00,2005-03-01 00:00:00,2005-03-02 00:00:00,2005-03-03 00:00:00,2005-03-04 00:00:00,2005-03-07 00:00:00,2005-03-08 00:00:00,2005-03-09 00:00:00,...,2020-05-07 00:00:00,2020-05-08 00:00:00,2020-05-11 00:00:00,2020-05-12 00:00:00,2020-05-13 00:00:00,2020-05-14 00:00:00,2020-05-15 00:00:00,2020-05-18 00:00:00,2020-05-19 00:00:00,2020-05-20 00:00:00,2020-05-21 00:00:00,2020-05-22 00:00:00,2020-05-25 00:00:00,2020-05-26 00:00:00,2020-05-27 00:00:00,2020-05-28 00:00:00,2020-05-29 00:00:00,2020-06-01 00:00:00,2020-06-02 00:00:00,2020-06-03 00:00:00,2020-06-04 00:00:00,2020-06-05 00:00:00,2020-06-08 00:00:00,2020-06-09 00:00:00,2020-06-10 00:00:00,2020-06-11 00:00:00,2020-06-12 00:00:00,2020-06-15 00:00:00,2020-06-16 00:00:00,2020-06-17 00:00:00,2020-06-18 00:00:00,2020-06-19 00:00:00,2020-06-22 00:00:00,2020-06-23 00:00:00,2020-06-24 00:00:00,2020-06-29 00:00:00,2020-06-30 00:00:00,2020-07-01 00:00:00,2020-07-02 00:00:00,2020-07-03 00:00:00
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
600089,12.00,11.94,11.91,11.37,11.60,11.70,11.78,11.57,11.50,11.37,11.11,11.10,10.59,10.90,10.90,11.00,10.98,10.73,10.28,10.05,10.00,10.60,10.59,10.82,10.70,10.95,10.95,11.27,11.54,11.40,11.18,11.20,10.85,10.88,10.93,10.78,10.49,10.59,10.85,10.06,...,7.600000,7.610000,7.780000,7.640000,7.860000,7.700000,7.630000,7.650000,7.580000,7.540000,7.200000,7.090000,7.130000,7.190000,7.070000,6.930000,6.870000,7.060000,7.120000,7.070000,7.020000,7.050000,7.020000,7.060000,6.930000,6.890000,6.82,6.710000,6.850000,6.820000,6.910000,6.910000,6.920000,6.880000,6.870000,6.700000,6.770000,6.990000,7.120000,7.180000
600104,4.60,4.65,4.61,4.78,4.82,4.76,4.74,4.73,4.85,4.75,4.93,5.01,5.11,5.29,5.35,5.58,5.50,5.10,5.09,5.08,5.14,5.65,5.65,5.83,5.69,5.60,5.55,5.67,5.75,5.60,5.63,5.56,5.49,5.40,5.32,5.29,5.21,5.10,5.22,5.32,...,19.650000,20.290001,20.100000,19.790001,19.520000,18.940001,18.600000,18.480000,18.709999,18.610001,18.510000,18.360001,18.750000,18.790001,18.230000,18.180000,17.920000,18.299999,18.230000,18.940001,18.230000,18.230000,18.299999,18.299999,18.110001,18.049999,17.99,17.719999,17.799999,17.870001,17.840000,18.250000,18.120001,17.969999,18.190001,17.980000,16.990000,17.250000,17.860001,18.650000
600109,6.15,6.26,6.20,6.30,6.33,6.34,6.28,6.32,6.24,6.00,6.08,6.04,5.90,5.99,6.12,6.05,5.99,5.91,5.75,5.55,5.47,5.74,5.60,5.69,5.78,5.81,5.75,5.85,6.00,6.15,6.11,6.04,6.05,6.35,6.09,6.15,6.04,6.10,6.18,6.25,...,9.820000,10.200000,10.250000,10.160000,10.210000,10.110000,10.060000,10.130000,10.130000,10.010000,10.020000,9.650000,9.660000,9.850000,9.710000,10.000000,9.840000,10.820000,11.030000,10.770000,10.780000,10.850000,10.430000,10.600000,10.510000,10.370000,10.35,10.320000,10.770000,10.680000,10.780000,11.190000,11.520000,11.720000,11.520000,11.040000,11.410000,11.530000,12.680000,13.950000
600111,6.00,6.06,6.05,6.12,6.12,6.18,6.12,6.13,6.06,5.82,5.85,5.75,5.71,5.84,5.99,5.89,5.81,5.68,5.67,5.31,5.26,5.61,5.41,5.50,5.54,5.64,5.61,6.10,6.18,6.08,6.10,6.06,6.27,6.41,6.15,6.31,6.19,6.20,6.34,6.33,...,9.280000,9.330000,9.250000,9.170000,9.200000,9.380000,9.390000,9.880000,9.810000,9.640000,9.850000,10.020000,9.840000,9.900000,10.040000,10.080000,10.150000,10.100000,10.080000,9.800000,9.770000,9.680000,9.730000,9.600000,9.670000,9.890000,9.69,9.390000,9.510000,9.460000,9.480000

In [ ]:
pn

<class 'pandas.core.panel.Panel'>
Dimensions: 4 (items) x 11 (major_axis) x 3766 (minor_axis)
Items axis: open to low
Major_axis axis: 600089 to 600518
Minor_axis axis: 2005-01-04 00:00:00 to 2020-07-03 00:00:00

##2.定义ReplayBuffer

In [ ]:
#定义experience和模型训练中进行下一个batch的训练操作
class ReplayBuffer:
    def __init__(self, start_index, end_index, batch_size, is_permed, coin_number, sample_bias=1.0):
        """
        :param start_index: start index of the training set on the global data matrices
        :param end_index: end index of the training set on the global data matrices
        """
        self.__coin_number = coin_number
        self.__experiences = [Experience(i) for i in range(start_index, end_index)]
        self.__is_permed = is_permed
        # NOTE: in order to achieve the previous w feature
        self.__batch_size = batch_size
        self.__sample_bias = sample_bias
        logging.debug("buffer_bias is %f" % sample_bias)

    def append_experience(self, state_index):
        self.__experiences.append(Experience(state_index))
        logging.debug("a new experience, indexed by %d, was appended" % state_index)

    def __sample(self, start, end, bias):
        """
        @:param end: is excluded
        @:param bias: value in (0, 1)
        """
        # TODO: deal with the case when bias is 0
        ran = np.random.geometric(bias)
        while ran > end - start:
            ran = np.random.geometric(bias)
        result = end - ran
        return result

    def next_experience_batch(self):
        # First get a start point randomly
        batch = []
        if self.__is_permed:
            for i in range(self.__batch_size):
                batch.append(self.__experiences[self.__sample(self.__experiences[0].state_index,
                                                              self.__experiences[-1].state_index,
                                                              self.__sample_bias)])
        else:
            batch_start = self.__sample(0, len(self.__experiences)-self.__batch_size,self.__sample_bias)
            batch = self.__experiences[batch_start:batch_start+self.__batch_size]
        return batch


class Experience:
    def __init__(self, state_index):
        self.state_index = int(state_index)

##3.数据矩阵处理

In [ ]:
#将数据转化成矩阵的形式便于训练，包括将模型分为训练集和测试集等
class DataMatrices:
    def __init__(self, start, end, period, batch_size=50, volume_average_days=30, buffer_bias_ratio=0,
                 market="poloniex", coin_filter=1, window_size=50, feature_number=3, test_portion=0.15,
                 portion_reversed=False, online=False, is_permed=False):
        """
        :param start: Unix time
        :param end: Unix time
        :param access_period: the data access period of the input matrix.
        :param trade_period: the trading period of the agent.
        :param global_period: the data access period of the global price matrix, if it is not equal to the access period, there will be inserted observations
        :param coin_filter: number of coins that would be selected
        :param window_size: periods of input data
        :param train_portion: portion of training set
        :param is_permed: if False, the sample inside a mini-batch is in order
        :param validation_portion: portion of cross-validation set
        :param test_portion: portion of test set
        :param portion_reversed: if False, the order to sets are [train, validation, test]
        else the order is [test, validation, train]
        """
        start = int(start)
        self.__end = int(end)

        # assert window_size >= MIN_NUM_PERIOD
        self.__coin_no = coin_filter
        self.__features = feature_list
        self.feature_number = feature_number
        # = get_volume_forward(self.__end-start, test_portion, portion_reversed)
        self.__global_data = panel_fillna(pn, "both")
        self.__period_length = period
        # portfolio vector memory, [time, assets]
        self.__PVM = pd.DataFrame(index=self.__global_data.minor_axis,columns=self.__global_data.major_axis)
        #初始设置等权重
        self.__PVM = self.__PVM.fillna(1.0 / self.__coin_no)

        self._window_size = window_size
        self._num_periods = len(self.__global_data.minor_axis)
        self.__divide_data(test_portion, portion_reversed)

        self._portion_reversed = portion_reversed
        self.__is_permed = is_permed

        self.__batch_size = batch_size
        self.__delta = 0  # the count of global increased
        end_index = self._train_ind[-1]
        self.__replay_buffer = ReplayBuffer(start_index=self._train_ind[0],end_index=end_index,
                            sample_bias=buffer_bias_ratio,batch_size=self.__batch_size,
                            coin_number=self.__coin_no,is_permed=self.__is_permed)

        logging.info("the number of training examples is %s"
                     ", of test examples is %s" % (self._num_train_samples, self._num_test_samples))
        logging.debug("the training set is from %s to %s" % (min(self._train_ind), max(self._train_ind)))
        logging.debug("the test set is from %s to %s" % (min(self._test_ind), max(self._test_ind)))

    @property
    def global_weights(self):
        return self.__PVM

    @staticmethod
    def create_from_config(config):
        """main method to create the DataMatrices in this project
        @:param config: config dictionary
        @:return: a DataMatrices object
        """
        config = config.copy()
        input_config = config["input"]
        train_config = config["training"]
        start = parse_time(input_config["start_date"])
        end = parse_time(input_config["end_date"])
        return DataMatrices(start=start,end=end,market=input_config["market"],feature_number=input_config["feature_number"],
                   window_size=input_config["window_size"],online=input_config["online"],
                   period=input_config["global_period"],coin_filter=input_config["coin_number"],
                   is_permed=input_config["is_permed"],buffer_bias_ratio=train_config["buffer_biased"],
                   batch_size=train_config["batch_size"],volume_average_days=input_config["volume_average_days"],
                   test_portion=input_config["test_portion"],portion_reversed=input_config["portion_reversed"],)

    @property
    def global_matrix(self):
        return self.__global_data

    @property
    def coin_list(self):
        return stock_ind

    @property
    def num_train_samples(self):
        return self._num_train_samples

    @property
    def test_indices(self):
        return self._test_ind[:-(self._window_size+1):]

    @property
    def num_test_samples(self):
        return self._num_test_samples

    def append_experience(self, online_w=None):
        """
        :param online_w: (number of assets + 1, ) numpy array
        Let it be None if in the backtest case.
        """
        self.__delta += 1
        self._train_ind.append(self._train_ind[-1]+1)
        appended_index = self._train_ind[-1]
        self.__replay_buffer.append_experience(appended_index)

    def get_test_set(self):
        return self.__pack_samples(self.test_indices)

    def get_training_set(self):
        return self.__pack_samples(self._train_ind[:-self._window_size])

    def next_batch(self):
        """
        @:return: the next batch of training sample. The sample is a dictionary
        with key "X"(input data); "y"(future relative price); "last_w" a numpy array
        with shape [batch_size, assets]; "w" a list of numpy arrays list length is
        batch_size
        """
        batch = self.__pack_samples([exp.state_index for exp in self.__replay_buffer.next_experience_batch()])
        return batch

    def __pack_samples(self, indexs):
        indexs = np.array(indexs)
        last_w = self.__PVM.values[indexs-1, :]

        def setw(w):
            self.__PVM.iloc[indexs, :] = w
        M = [self.get_submatrix(index) for index in indexs]
        M = np.array(M)
        X = M[:, :, :, :-1]
        y = M[:, :, :, -1] / M[:, 0, None, :, -2]
        return {"X": X, "y": y, "last_w": last_w, "setw": setw}

    # volume in y is the volume in next access period
    def get_submatrix(self, ind):
        return self.__global_data.values[:, :, ind:ind+self._window_size+1]

    def __divide_data(self, test_portion, portion_reversed):
        train_portion = 1 - test_portion
        s = float(train_portion + test_portion)
        if portion_reversed:
            portions = np.array([test_portion]) / s
            portion_split = (portions * self._num_periods).astype(int)
            indices = np.arange(self._num_periods)
            self._test_ind, self._train_ind = np.split(indices, portion_split)
        else:
            portions = np.array([train_portion]) / s
            portion_split = (portions * self._num_periods).astype(int)
            indices = np.arange(self._num_periods)
            self._train_ind, self._test_ind = np.split(indices, portion_split)

        self._train_ind = self._train_ind[:-(self._window_size + 1)]
        # change the logic here in order to fit both reversed and normal version
        self._train_ind = list(self._train_ind)
        self._num_train_samples = len(self._train_ind)
        self._num_test_samples = len(self.test_indices)


#三、神经网络训练及回测

##1.定义神经网络

In [ ]:
#定义父类，初始化神经网络
class NeuralNetWork:
    def __init__(self, feature_number, rows, columns, layers, device):
        tf_config = tf.ConfigProto()
        self.session = tf.Session(config=tf_config)
        if device == "cpu":
            tf_config.gpu_options.per_process_gpu_memory_fraction = 0
        else:
            tf_config.gpu_options.per_process_gpu_memory_fraction = 0.2  #占用20%显存
        self.input_num = tf.placeholder(tf.int32, shape=[])
        self.input_tensor = tf.placeholder(tf.float32, shape=[None, feature_number, rows, columns])
        self.previous_w = tf.placeholder(tf.float32, shape=[None, rows])
        self._rows = rows
        self._columns = columns

        self.layers_dict = {}
        self.layer_count = 0

        self.output = self._build_network(layers)

    def _build_network(self, layers):
        pass

In [ ]:
#定义模型使用的神经网络，根据config选择对应的模型（CNN、bRNN、LSTM）
#应用tflearn进行神经网络的初始化以及模型的搭建
class CNN(NeuralNetWork):
    # input_shape (features, rows, columns)
    def __init__(self, feature_number, rows, columns, layers, device):
        NeuralNetWork.__init__(self, feature_number, rows, columns, layers, device)

    def add_layer_to_dict(self, layer_type, tensor, weights=True):

        self.layers_dict[layer_type + '_' + str(self.layer_count) + '_activation'] = tensor
        self.layer_count += 1

    # generate the operation, the forward computation
    def _build_network(self, layers):
        network = tf.transpose(self.input_tensor, [0, 2, 3, 1])
        # [batch, assets, window, features]
        network = network / network[:, :, -1, 0, None, None]
        for layer_number, layer in enumerate(layers):
            if layer["type"] == "DenseLayer":
                network = tflearn.layers.core.fully_connected(network,
                                        int(layer["neuron_number"]),
                                        layer["activation_function"],
                                        regularizer=layer["regularizer"],
                                        weight_decay=layer["weight_decay"] )
                self.add_layer_to_dict(layer["type"], network)
            elif layer["type"] == "DropOut":
                network = tflearn.layers.core.dropout(network, layer["keep_probability"])
            elif layer["type"] == "EIIE_Dense":
                width = network.get_shape()[2]
                network = tflearn.layers.conv_2d(network, int(layer["filter_number"]),
                                  [1, width],[1, 1],"valid",
                                  layer["activation_function"],
                                  regularizer=layer["regularizer"],
                                  weight_decay=layer["weight_decay"])
                self.add_layer_to_dict(layer["type"], network)
            elif layer["type"] == "ConvLayer":
                network = tflearn.layers.conv_2d(network, int(layer["filter_number"]),
                                  allint(layer["filter_shape"]),
                                  allint(layer["strides"]),
                                  layer["padding"],
                                  layer["activation_function"],
                                  regularizer=layer["regularizer"],
                                  weight_decay=layer["weight_decay"])
                self.add_layer_to_dict(layer["type"], network)
            elif layer["type"] == "MaxPooling":
                network = tflearn.layers.conv.max_pool_2d(network, layer["strides"])
            elif layer["type"] == "AveragePooling":
                network = tflearn.layers.conv.avg_pool_2d(network, layer["strides"])
            elif layer["type"] == "LocalResponseNormalization":
                network = tflearn.layers.normalization.local_response_normalization(network)
            elif layer["type"] == "EIIE_Output":
                width = network.get_shape()[2]
                network = tflearn.layers.conv_2d(network, 1, [1, width], padding="valid",
                                                 regularizer=layer["regularizer"],
                                                 weight_decay=layer["weight_decay"])
                self.add_layer_to_dict(layer["type"], network)
                network = network[:, :, 0, 0]
                btc_bias = tf.ones((self.input_num, 1))
                self.add_layer_to_dict(layer["type"], network)
                network = tf.concat([btc_bias, network], 1)
                network = tflearn.layers.core.activation(network, activation="softmax")
                self.add_layer_to_dict(layer["type"], network, weights=False)
            elif layer["type"] == "Output_WithW":
                network = tflearn.flatten(network)
                network = tf.concat([network,self.previous_w], axis=1)
                network = tflearn.fully_connected(network, self._rows+1,
                                                  activation="softmax",
                                                  regularizer=layer["regularizer"],
                                                  weight_decay=layer["weight_decay"])
            elif layer["type"] == "EIIE_Output_WithW":
                width = network.get_shape()[2]
                height = network.get_shape()[1]
                features = network.get_shape()[3]
                network = tf.reshape(network, [self.input_num, int(height), 1, int(width*features)])
                w = tf.reshape(self.previous_w, [-1, int(height), 1, 1])
                network = tf.concat([network, w], axis=3)
                network = tflearn.layers.conv_2d(network, 1, [1, 1], padding="valid",
                                  regularizer=layer["regularizer"],
                                  weight_decay=layer["weight_decay"])
                self.add_layer_to_dict(layer["type"], network)
                network = network[:, :, 0, 0]
                #btc_bias = tf.zeros((self.input_num, 1))
                btc_bias = tf.get_variable("btc_bias", [1, 1], dtype=tf.float32,
                                       initializer=tf.zeros_initializer)
                # self.add_layer_to_dict(layer["type"], network, weights=False)
                btc_bias = tf.tile(btc_bias, [self.input_num, 1])
                network = tf.concat([btc_bias, network], 1)
                self.voting = network
                self.add_layer_to_dict('voting', network, weights=False)
                network = tflearn.layers.core.activation(network, activation="softmax")
                self.add_layer_to_dict('softmax_layer', network, weights=False)

            elif layer["type"] == "EIIE_LSTM" or layer["type"] == "EIIE_RNN":
                network = tf.transpose(network, [0, 2, 3, 1])
                resultlist = []
                reuse = False
                for i in range(self._rows):
                    if i > 0:
                        reuse = True
                    if layer["type"] == "EIIE_LSTM":
                        result = tflearn.layers.lstm(network[:, :, :, i],
                                        int(layer["neuron_number"]),
                                        dropout=layer["dropouts"],
                                        scope="lstm"+str(layer_number),
                                        reuse=reuse)
                    else:
                        result = tflearn.layers.simple_rnn(network[:, :, :, i],
                                           int(layer["neuron_number"]),
                                           dropout=layer["dropouts"],
                                           scope="rnn"+str(layer_number),
                                           reuse=reuse)
                    resultlist.append(result)
                network = tf.stack(resultlist)
                network = tf.transpose(network, [1, 0, 2])
                network = tf.reshape(network, [-1, self._rows, 1, int(layer["neuron_number"])])
            else:
                raise ValueError("the layer {} not supported.".format(layer["type"]))
        return network
def allint(l):
    return [int(i) for i in l]

##2.定义agent

In [ ]:
#建立模型的agent
class NNAgent:
    def __init__(self, config, restore_dir=None, device="cpu"):
        self.__config = config
        self.__coin_number = config["input"]["coin_number"]
        self.__net = CNN(config["input"]["feature_number"],
                  self.__coin_number,
                  config["input"]["window_size"],
                  config["layers"],
                  device=device)
        self.__global_step = tf.Variable(0, trainable=False)
        self.__train_operation = None
        self.__y = tf.placeholder(tf.float32, shape=[None,
                                 self.__config["input"]["feature_number"],
                                 self.__coin_number])
        self.__future_price = tf.concat([tf.ones([self.__net.input_num, 1]),
                                       self.__y[:, 0, :]], 1)
        self.__future_omega = (self.__future_price * self.__net.output) /\
                     tf.reduce_sum(self.__future_price * self.__net.output, axis=1)[:, None]
        # tf.assert_equal(tf.reduce_sum(self.__future_omega, axis=1), tf.constant(1.0))
        self.__commission_ratio = self.__config["trading"]["trading_consumption"]
        self.__pv_vector = tf.reduce_sum(self.__net.output * self.__future_price, reduction_indices=[1]) *\
                           (tf.concat([tf.ones(1), self.__pure_pc()], axis=0))
        self.__log_mean_free = tf.reduce_mean(tf.log(tf.reduce_sum(self.__net.output * self.__future_price,
                                                                   reduction_indices=[1])))
        self.__portfolio_value = tf.reduce_prod(self.__pv_vector)
        self.__mean = tf.reduce_mean(self.__pv_vector)
        self.__log_mean = tf.reduce_mean(tf.log(self.__pv_vector))
        self.__standard_deviation = tf.sqrt(tf.reduce_mean((self.__pv_vector - self.__mean) ** 2))
        self.__sharp_ratio = (self.__mean - 1) / self.__standard_deviation
        self.__loss = self.__set_loss_function()
        self.__train_operation = self.init_train(learning_rate=self.__config["training"]["learning_rate"],
                               decay_steps=self.__config["training"]["decay_steps"],
                               decay_rate=self.__config["training"]["decay_rate"],
                               training_method=self.__config["training"]["training_method"])
        self.__saver = tf.train.Saver()
        if restore_dir:
            self.__saver.restore(self.__net.session, restore_dir)
        else:
            self.__net.session.run(tf.global_variables_initializer())

    @property
    def session(self):
        return self.__net.session

    @property
    def pv_vector(self):
        return self.__pv_vector

    @property
    def standard_deviation(self):
        return self.__standard_deviation

    @property
    def portfolio_weights(self):
        return self.__net.output

    @property
    def sharp_ratio(self):
        return self.__sharp_ratio

    @property
    def log_mean(self):
        return self.__log_mean

    @property
    def log_mean_free(self):
        return self.__log_mean_free

    @property
    def portfolio_value(self):
        return self.__portfolio_value

    @property
    def loss(self):
        return self.__loss

    @property
    def layers_dict(self):
        return self.__net.layers_dict

    def recycle(self):
        tf.reset_default_graph()
        self.__net.session.close()

    def __set_loss_function(self):
        LAMBDA = 1e-4
        def loss_function4():
            return -tf.reduce_mean(tf.log(tf.reduce_sum(self.__net.output[:] * self.__future_price,
                                                        reduction_indices=[1])))

        def loss_function5():
            return -tf.reduce_mean(tf.log(tf.reduce_sum(self.__net.output * self.__future_price, reduction_indices=[1]))) + \
                   LAMBDA * tf.reduce_mean(tf.reduce_sum(-tf.log(1 + 1e-6 - self.__net.output), reduction_indices=[1]))

        def loss_function6():
            return -tf.reduce_mean(tf.log(self.pv_vector))

        def loss_function7():
            return -tf.reduce_mean(tf.log(self.pv_vector)) + \
                   LAMBDA * tf.reduce_mean(tf.reduce_sum(-tf.log(1 + 1e-6 - self.__net.output), reduction_indices=[1]))

        def with_last_w():
            return -tf.reduce_mean(tf.log(tf.reduce_sum(self.__net.output[:] * self.__future_price, reduction_indices=[1])
                                          -tf.reduce_sum(tf.abs(self.__net.output[:, 1:] - self.__net.previous_w)
                                                         *self.__commission_ratio, reduction_indices=[1])))

        loss_function = loss_function5
        if self.__config["training"]["loss_function"] == "loss_function4":
            loss_function = loss_function4
        elif self.__config["training"]["loss_function"] == "loss_function5":
            loss_function = loss_function5
        elif self.__config["training"]["loss_function"] == "loss_function6":
            loss_function = loss_function6
        elif self.__config["training"]["loss_function"] == "loss_function7":
            loss_function = loss_function7
        elif self.__config["training"]["loss_function"] == "loss_function8":
            loss_function = with_last_w

        loss_tensor = loss_function()
        regularization_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
        if regularization_losses:
            for regularization_loss in regularization_losses:
                loss_tensor += regularization_loss
        return loss_tensor

    def init_train(self, learning_rate, decay_steps, decay_rate, training_method):
        learning_rate = tf.train.exponential_decay(learning_rate, self.__global_step,
                                decay_steps, decay_rate, staircase=True)
        if training_method == 'GradientDescent':
            train_step = tf.train.GradientDescentOptimizer(learning_rate).\
                         minimize(self.__loss, global_step=self.__global_step)
        elif training_method == 'Adam':
            train_step = tf.train.AdamOptimizer(learning_rate).\
                         minimize(self.__loss, global_step=self.__global_step)
        elif training_method == 'RMSProp':
            train_step = tf.train.RMSPropOptimizer(learning_rate).\
                         minimize(self.__loss, global_step=self.__global_step)
        else:
            raise ValueError()
        return train_step

    def train(self, x, y, last_w, setw):
        tflearn.is_training(True, self.__net.session)
        self.evaluate_tensors(x, y, last_w, setw, [self.__train_operation])

    def evaluate_tensors(self, x, y, last_w, setw, tensors):
        """
        :param x:
        :param y:
        :param last_w:
        :param setw: a function, pass the output w to it to fill the PVM
        :param tensors:
        :return:
        """
        tensors = list(tensors)
        tensors.append(self.__net.output)
        assert not np.any(np.isnan(x))
        assert not np.any(np.isnan(y))
        assert not np.any(np.isnan(last_w)),\
            "the last_w is {}".format(last_w)
        results = self.__net.session.run(tensors,
                          feed_dict={self.__net.input_tensor: x,
                                self.__y: y,
                                self.__net.previous_w: last_w,
                                self.__net.input_num: x.shape[0]})
        setw(results[-1][:, 1:])
        return results[:-1]

    # save the variables path including file name
    def save_model(self, path):
        self.__saver.save(self.__net.session, path)

    # consumption vector (on each periods)
    def __pure_pc(self):
        c = self.__commission_ratio
        w_t = self.__future_omega[:self.__net.input_num-1]  # rebalanced
        w_t1 = self.__net.output[1:self.__net.input_num]
        mu = 1 - tf.reduce_sum(tf.abs(w_t1[:, 1:]-w_t[:, 1:]), axis=1)*c
        return mu

    # the history is a 3d matrix, return a asset vector
    def decide_by_history(self, history, last_w):
        assert isinstance(history, np.ndarray),\
            "the history should be a numpy array, not %s" % type(history)
        assert not np.any(np.isnan(last_w))
        assert not np.any(np.isnan(history))
        tflearn.is_training(False, self.session)
        history = history[np.newaxis, :, :, :]
        return np.squeeze(self.session.run(self.__net.output, feed_dict={self.__net.input_tensor: history,
                                                                         self.__net.previous_w: last_w[np.newaxis, 1:],
                                                                         self.__net.input_num: 1}))

##3.主训练函数

In [ ]:
def calculate_upperbound(y):
  array = np.maximum.reduce(y[:, 0, :], 1)
  total = 1.0
  for i in array:
      total = total * i
  return total

In [ ]:
Result = collections.namedtuple("Result",
                                [
                                 "test_pv",
                                 "test_log_mean",
                                 "test_log_mean_free",
                                 "test_history",
                                 "config",
                                 "net_dir",
                                 "backtest_test_pv",
                                 "backtest_test_history",
                                 "backtest_test_log_mean",
                                 "training_time"])

In [ ]:
#定义了主训练函数train_net、用于输出和保存结果的log_result_csv等

class TraderTrainer:
    def __init__(self, config, fake_data=False, restore_dir=None, save_path=None, device="cpu",
                 agent=None):
        """
        :param config: config dictionary
        :param fake_data: if True will use data generated randomly
        :param restore_dir: path to the model trained before
        :param save_path: path to save the model
        :param device: the device used to train the network
        :param agent: the nnagent object. If this is provides, the trainer will not
        create a new agent by itself. Therefore the restore_dir will not affect anything.
        """
        self.config = config
        self.train_config = config["training"]
        self.input_config = config["input"]
        self.save_path = save_path
        self.best_metric = 0
        np.random.seed(config["random_seed"])

        self.__window_size = self.input_config["window_size"]
        self.__coin_number = self.input_config["coin_number"]
        self.__batch_size = self.train_config["batch_size"]
        self.__snap_shot = self.train_config["snap_shot"]
        config["input"]["fake_data"] = fake_data

        self._matrix = DataMatrices.create_from_config(config)

        self.test_set = self._matrix.get_test_set()
        if not config["training"]["fast_train"]:
            self.training_set = self._matrix.get_training_set()
        self.upperbound_validation = 1
        self.upperbound_test = 1
        tf.set_random_seed(self.config["random_seed"])
        self.device = device
        if agent:
            self._agent = agent
        else:
            if device == "cpu":
                os.environ["CUDA_VISIBLE_DEVICES"] = ""
                with tf.device("/cpu:0"):
                    self._agent = NNAgent(config, restore_dir, device)
            else:
                self._agent = NNAgent(config, restore_dir, device)

    def _evaluate(self, set_name, *tensors):
        if set_name == "test":
            feed = self.test_set
        elif set_name == "training":
            feed = self.training_set
        else:
            raise ValueError()
        result = self._agent.evaluate_tensors(feed["X"],feed["y"],last_w=feed["last_w"],
                             setw=feed["setw"], tensors=tensors)
        return result

    @staticmethod
    def calculate_upperbound(y):
        array = np.maximum.reduce(y[:, 0, :], 1)
        total = 1.0
        for i in array:
            total = total * i
        return total

    def log_between_steps(self, step):
        fast_train = self.train_config["fast_train"]
        tflearn.is_training(False, self._agent.session)

        summary, v_pv, v_log_mean, v_loss, log_mean_free, weights= \
            self._evaluate("test", self.summary,
                           self._agent.portfolio_value,
                           self._agent.log_mean,
                           self._agent.loss,
                           self._agent.log_mean_free,
                           self._agent.portfolio_weights)
        self.test_writer.add_summary(summary, step)

        if not fast_train:
            summary, loss_value = self._evaluate("training", self.summary, self._agent.loss)
            self.train_writer.add_summary(summary, step)

        # print 'ouput is %s' % out
        logging.info('='*30)
        logging.info('step %d' % step)
        logging.info('-'*30)
        if not fast_train:
            logging.info('training loss is %s\n' % loss_value)
        logging.info('the portfolio value on test set is %s\nlog_mean is %s\n'
                     'loss_value is %3f\nlog mean without commission fee is %3f\n' % \
                     (v_pv, v_log_mean, v_loss, log_mean_free))
        logging.info('='*30+"\n")

        if not self.__snap_shot:
            self._agent.save_model(self.save_path)
        elif v_pv > self.best_metric:
            self.best_metric = v_pv
            logging.info("get better model at %s steps,"
                         " whose test portfolio value is %s" % (step, v_pv))
            if self.save_path:
                self._agent.save_model(self.save_path)
        self.check_abnormal(v_pv, weights)

    def check_abnormal(self, portfolio_value, weigths):
        if portfolio_value == 1.0:
            logging.info("average portfolio weights {}".format(weigths.mean(axis=0)))


    def next_batch(self):
        batch = self._matrix.next_batch()
        batch_input = batch["X"]
        batch_y = batch["y"]
        batch_last_w = batch["last_w"]
        batch_w = batch["setw"]
        return batch_input, batch_y, batch_last_w, batch_w

    def __init_tensor_board(self, log_file_dir):
        tf.summary.scalar('benefit', self._agent.portfolio_value)
        tf.summary.scalar('log_mean', self._agent.log_mean)
        tf.summary.scalar('loss', self._agent.loss)
        tf.summary.scalar("log_mean_free", self._agent.log_mean_free)
        for layer_key in self._agent.layers_dict:
            tf.summary.histogram(layer_key, self._agent.layers_dict[layer_key])
        for var in tf.trainable_variables():
            tf.summary.histogram(var.name, var)
        grads = tf.gradients(self._agent.loss, tf.trainable_variables())
        for grad in grads:
            tf.summary.histogram(grad.name + '/gradient', grad)
        self.summary = tf.summary.merge_all()
        location = log_file_dir
        self.network_writer = tf.summary.FileWriter(location + '/network',
                                                    self._agent.session.graph)
        self.test_writer = tf.summary.FileWriter(location + '/test')
        self.train_writer = tf.summary.FileWriter(location + '/train')

    def __print_upperbound(self):
        upperbound_test = self.calculate_upperbound(self.test_set["y"])
        logging.info("upper bound in test is %s" % upperbound_test)

    def train_net(self, log_file_dir="./tensorboard", index="0"):
        """
        :param log_file_dir: logging of the training process
        :param index: sub-folder name under train_package
        :return: the result named tuple
        """
        print(self.test_set)
        self.__print_upperbound()
        if log_file_dir:
            if self.device == "cpu":
                with tf.device("/cpu:0"):
                    self.__init_tensor_board(log_file_dir)
            else:
                self.__init_tensor_board(log_file_dir)
        starttime = time.time()

        total_data_time = 0
        total_training_time = 0
        for i in range(self.train_config["steps"]):
            step_start = time.time()
            x, y, last_w, setw = self.next_batch()
            finish_data = time.time()
            total_data_time += (finish_data - step_start)
            self._agent.train(x, y, last_w=last_w, setw=setw)
            total_training_time += time.time() - finish_data
            if i % 1000 == 0 and log_file_dir:
                logging.info("average time for data accessing is %s"%(total_data_time/1000))
                logging.info("average time for training is %s"%(total_training_time/1000))
                total_training_time = 0
                total_data_time = 0
                self.log_between_steps(i)

        if self.save_path:
            self._agent.recycle()
            best_agent = NNAgent(self.config, restore_dir=self.save_path)
            self._agent = best_agent

        pv, log_mean = self._evaluate("test", self._agent.portfolio_value, self._agent.log_mean)
        logging.warning('the portfolio value train No.%s is %s log_mean is %s,'
                        ' the training time is %d seconds' % (index, pv, log_mean, time.time() - starttime))

        return self.__log_result_csv(index, time.time() - starttime)

    def __log_result_csv(self, index, time):
        dataframe = None
        csv_dir = './train_package/train_summary.csv'
        tflearn.is_training(False, self._agent.session)
        v_pv, v_log_mean, benefit_array, v_log_mean_free =\
            self._evaluate("test",
                           self._agent.portfolio_value,
                           self._agent.log_mean,
                           self._agent.pv_vector,
                           self._agent.log_mean_free)

        backtest = BackTest(self.config.copy(),
                                     net_dir=None,
                                     agent=self._agent)

        backtest.start_trading()
        result = Result(test_pv=[v_pv],
                        test_log_mean=[v_log_mean],
                        test_log_mean_free=[v_log_mean_free],
                        test_history=[''.join(str(e)+', ' for e in benefit_array)],
                        config=[json.dumps(self.config)],
                        net_dir=[index],
                        backtest_test_pv=[backtest.test_pv],
                        backtest_test_history=[''.join(str(e)+', ' for e in backtest.test_pc_vector)],
                        backtest_test_log_mean=[np.mean(np.log(backtest.test_pc_vector))],
                        training_time=int(time))
        new_data_frame = pd.DataFrame(result._asdict()).set_index("net_dir")
        if os.path.isfile(csv_dir):
            dataframe = pd.read_csv(csv_dir).set_index("net_dir")
            dataframe = dataframe.append(new_data_frame)
        else:
            dataframe = new_data_frame
        if int(index) > 0:
            dataframe.to_csv(csv_dir)
        return result


##4.滚动训练函数

In [ ]:
#定义用于在回测阶段做滚动训练的函数
class RollingTrainer(TraderTrainer):
    def __init__(self, config, restore_dir=None, save_path=None, agent=None, device="cpu"):
        config["training"]["buffer_biased"] = config["trading"]["buffer_biased"]
        config["training"]["learning_rate"] = config["trading"]["learning_rate"]
        TraderTrainer.__init__(self, config, restore_dir=restore_dir, save_path=save_path,
                               agent=agent, device=device)

    @property
    def agent(self):
        return self._agent

    @property
    def coin_list(self):
        return self._matrix.coin_list

    @property
    def data_matrices(self):
        return self._matrix

    @property
    def rolling_training_steps(self):
        return self.config["trading"]["rolling_training_steps"]

    def __rolling_logging(self):
        fast_train = self.train_config["fast_train"]
        if not fast_train:
            tflearn.is_training(False, self._agent.session)

            v_pv, v_log_mean = self._evaluate("validation",
                                              self._agent.portfolio_value,
                                              self._agent.log_mean)
            t_pv, t_log_mean = self._evaluate("test", self._agent.portfolio_value, self._agent.log_mean)
            loss_value = self._evaluate("training", self._agent.loss)

            logging.info('training loss is %s\n' % loss_value)
            logging.info('the portfolio value on validation asset is %s\nlog_mean is %s\n' %
                         (v_pv,v_log_mean))
            logging.info('the portfolio value on test asset is %s\n mean is %s' % (t_pv,t_log_mean))

    def decide_by_history(self, history, last_w):
        result = self._agent.decide_by_history(history, last_w)
        return result

    def rolling_train(self, online_w=None):
        steps = self.rolling_training_steps
        if steps > 0:
            self._matrix.append_experience(online_w)
            for i in range(steps):
                x, y, last_w, w = self.next_batch()
                self._agent.train(x, y, last_w, w)
            self.__rolling_logging()


##5.交易函数

In [ ]:
#应用模型结果进行模拟交易回测的函数
class Trader:
    def __init__(self, waiting_period, config, total_steps, net_dir, agent=None, initial_BTC=1.0, agent_type="nn"):
        """
        @:param agent_type: string, could be nn or traditional
        @:param agent: the traditional agent object, if the agent_type is traditional
        """
        self._steps = 0
        self._total_steps = total_steps
        self._period = waiting_period
        self._agent_type = agent_type

        if agent_type == "traditional":
            config["input"]["feature_number"] = 1
            config["input"]["norm_method"] = "relative"
            self._norm_method = "relative"
        elif agent_type == "nn":
            self._rolling_trainer = RollingTrainer(config, net_dir, agent=agent)
            self._coin_name_list = self._rolling_trainer.coin_list
            self._norm_method = config["input"]["norm_method"]
            if not agent:
                agent = self._rolling_trainer.agent
        else:
            raise ValueError()
        self._agent = agent

        # the total assets is calculated with BTC
        self._total_capital = initial_BTC
        self._window_size = config["input"]["window_size"]
        self._coin_number = config["input"]["coin_number"]
        self._commission_rate = config["trading"]["trading_consumption"]
        self._fake_ratio = config["input"]["fake_ratio"]
        self._asset_vector = np.zeros(self._coin_number+1)

        self._last_omega = np.zeros((self._coin_number+1,))
        self._last_omega[0] = 1.0

        if self.__class__.__name__=="BackTest":
            # self._initialize_logging_data_frame(initial_BTC)
            self._logging_data_frame = None
            # self._disk_engine =  sqlite3.connect('./database/back_time_trading_log.db')
            # self._initialize_data_base()
        self._current_error_state = 'S000'
        self._current_error_info = ''

    def _initialize_logging_data_frame(self, initial_BTC):
        logging_dict = {'Total Asset (RMB)': initial_BTC, 'RMB': 1}
        for coin in self._coin_name_list:
            logging_dict[coin] = 0
        self._logging_data_frame = pd.DataFrame(logging_dict, index=pd.to_datetime([time.time()], unit='s'))


    # add trading data into the pandas data frame
    def _log_trading_info(self, time, omega):
        time_index = pd.to_datetime([time], unit='s')
        if self._steps > 0:
            logging_dict = {'Total Asset (RMB)': self._total_capital, 'RMB': omega[0, 0]}
            for i in range(len(self._coin_name_list)):
                logging_dict[self._coin_name_list[i]] = omega[0, i + 1]
            new_data_frame = pd.DataFrame(logging_dict, index=time_index)
            self._logging_data_frame = self._logging_data_frame.append(new_data_frame)


    def __trade_body(self):
        self._current_error_state = 'S000'
        starttime = time.time()
        omega = self._agent.decide_by_history(self.generate_history_matrix(),
                                              self._last_omega.copy())
        self.trade_by_strategy(omega)
        if self._agent_type == "nn":
            self.rolling_train()
        if not self.__class__.__name__=="BackTest":
            self._last_omega = omega.copy()
        logging.info('total assets are %3f RMB' % self._total_capital)
        logging.debug("="*30)
        trading_time = time.time() - starttime
        if trading_time < self._period:
            logging.info("sleep for %s seconds" % (self._period - trading_time))
        self._steps += 1
        return self._period - trading_time

    def start_trading(self):
        try:
            if not self.__class__.__name__=="BackTest":
                current = int(time.time())
                wait = self._period - (current%self._period)
                logging.info("sleep for %s seconds" % wait)
                time.sleep(wait+2)

                while self._steps < self._total_steps:
                    sleeptime = self.__trade_body()
                    time.sleep(sleeptime)
            else:
                while self._steps < self._total_steps:
                    self.__trade_body()
        finally:
            if self._agent_type=="nn":
                self._agent.recycle()
            self.finish_trading()


##6.回测

In [ ]:
#定义回测函数并输出回测结果
class BackTest(Trader):
    def __init__(self, config, net_dir=None, agent=None, agent_type="nn"):
        Trader.__init__(self, 0, config, 0, net_dir, initial_BTC=1, agent=agent, agent_type=agent_type)
        if agent_type == "nn":
            data_matrices = self._rolling_trainer.data_matrices
        elif agent_type == "traditional":
            config["input"]["feature_number"] = 1
            data_matrices = DataMatrices.create_from_config(config)
        else:
            raise ValueError()
        self.__test_set = data_matrices.get_test_set()
        self.__test_length = self.__test_set["X"].shape[0]
        self._total_steps = self.__test_length
        self.__test_pv = 1.0
        self.__test_pc_vector = []

    @property
    def test_pv(self):
        return self.__test_pv

    @property
    def test_pc_vector(self):
        return np.array(self.__test_pc_vector, dtype=np.float32)

    def finish_trading(self):
        self.__test_pv = self._total_capital

    def __get_matrix_X(self):
        return self.__test_set["X"][self._steps]

    def __get_matrix_y(self):
        return self.__test_set["y"][self._steps, 0, :]

    def rolling_train(self, online_sample=None):
        self._rolling_trainer.rolling_train()

    def generate_history_matrix(self):
        inputs = self.__get_matrix_X()
        if self._agent_type == "traditional":
            inputs = np.concatenate([np.ones([1, 1, inputs.shape[2]]), inputs], axis=1)
            inputs = inputs[:, :, 1:] / inputs[:, :, :-1]
        return inputs

    def trade_by_strategy(self, omega):
        logging.info("the step is {}".format(self._steps))
        logging.info("the raw omega is {}".format(omega))
        future_price = np.concatenate((np.ones(1), self.__get_matrix_y()))
        pv_after_commission = calculate_pv_after_commission(omega, self._last_omega, self._commission_rate)
        portfolio_change = pv_after_commission * np.dot(omega, future_price)
        self._total_capital *= portfolio_change
        self._last_omega = pv_after_commission * omega * \
                           future_price /\
                           portfolio_change
        logging.info("the portfolio change this period is : {}".format(portfolio_change))
        self.__test_pc_vector.append(portfolio_change)


##7.执行训练的函数

In [ ]:
#用于模型训练，整合前面定义的函数
def train_one(save_path, config, log_file_dir, index, logfile_level, console_level, device):
    """
    train an agent
    :param save_path: the path to save the tensorflow model (.ckpt), could be None
    :param config: the json configuration file
    :param log_file_dir: the directory to save the tensorboard logging file, could be None
    :param index: identifier of this train, which is also the sub directory in the train_package,
    if it is 0. nothing would be saved into the summary file.
    :param logfile_level: logging level of the file
    :param console_level: logging level of the console
    :param device: 0 or 1 to show which gpu to use, if 0, means use cpu instead of gpu
    :return : the Result namedtuple
    """
    if log_file_dir:
        logging.basicConfig(filename=log_file_dir.replace("tensorboard","programlog"),
                            level=logfile_level)
        console = logging.StreamHandler()
        console.setLevel(console_level)
        logging.getLogger().addHandler(console)
    print("training at %s started" % index)
    return TraderTrainer(config, save_path=save_path, device=device).train_net(log_file_dir=log_file_dir, index=index)


In [ ]:
def train_all(processes=1, device="cpu"):
    """
    train all the agents in the train_package folders

    :param processes: the number of the processes. If equal to 1, the logging level is debug
                      at file and info at console. If greater than 1, the logging level is
                      info at file and warning at console.
    """
    if processes == 1:
        console_level = logging.INFO
        logfile_level = logging.DEBUG
    else:
        console_level = logging.WARNING
        logfile_level = logging.INFO
    train_dir = "train_package"
    if not os.path.exists("./" + train_dir): #if the directory does not exist, creates one
        os.makedirs("./" + train_dir)
    all_subdir = os.listdir("./" + train_dir)
    all_subdir.sort()
    pool = []
    for dir in all_subdir:
        # train only if the log dir does not exist
        if not str.isdigit(dir):
            return
        # NOTE: logfile is for compatibility reason
        if not (os.path.isdir("./"+train_dir+"/"+dir+"/tensorboard") or os.path.isdir("./"+train_dir+"/"+dir+"/logfile")):
            p = Process(target=train_one, args=(
                "./" + train_dir + "/" + dir + "/netfile",
                load_config(dir),
                "./" + train_dir + "/" + dir + "/tensorboard",
                dir, logfile_level, console_level, device))
            p.start()
            pool.append(p)
        else:
            continue

        # suspend if the processes are too many
        wait = True
        while wait:
            time.sleep(5)
            for p in pool:
                alive = p.is_alive()
                if not alive:
                    pool.remove(p)
            if len(pool)<processes:
                wait = False
    print("All the Tasks are Over")

#四、训练及回测

In [ ]:
train_all(1,'gpu')

training at 8 started


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

INFO:root:the number of training examples is 3432, of test examples is 270
the number of training examples is 3432, of test examples is 270



From /tensorflow-1.15.2/python3.6/tflearn/layers/conv.py:73: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
From /tensorflow-1.15.2/python3.6/tflearn/initializations.py:119: calling UniformUnitScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


...

INFO:tensorflow:Summary name gradients_1/Conv2D_2/BiasAdd_grad/BiasAddGrad:0/gradient is illegal; using gradients_1/Conv2D_2/BiasAdd_grad/BiasAddGrad_0/gradient instead.
Summary name gradients_1/Conv2D_2/BiasAdd_grad/BiasAddGrad:0/gradient is illegal; using gradients_1/Conv2D_2/BiasAdd_grad/BiasAddGrad_0/gradient instead.


INFO:tensorflow:Summary name gradients_1/Tile_grad/Sum:0/gradient is illegal; using gradients_1/Tile_grad/Sum_0/gradient instead.


INFO:tensorflow:Summary name gradients_1/Tile_grad/Sum:0/gradient is illegal; using gradients_1/Tile_grad/Sum_0/gradient instead.
Summary name gradients_1/Tile_grad/Sum:0/gradient is illegal; using gradients_1/Tile_grad/Sum_0/gradient instead.
INFO:root:average time for data accessing is 4.359960556030273e-06
average time for data accessing is 4.359960556030273e-06
INFO:root:average time for training is 0.00016357135772705078
average time for training is 0.00016357135772705078
INFO:root:==============================
INFO:root:step 0
step 0
INFO:root:------------------------------
------------------------------
INFO:root:the portfolio value on test set is 0.90136427
log_mean is -0.0003846131
loss_value is 0.000385
log mean without commission fee is -0.000373

the portfolio value on test set is 0.90136427
log_mean is -0.0003846131
loss_value is 0.000385
log mean without commission fee is -0.000373

INFO:root:==============================


INFO:root:average time for data accessing is 0

INFO:tensorflow:Restoring parameters from ./train_package/8/netfile


INFO:tensorflow:Restoring parameters from ./train_package/8/netfile
Restoring parameters from ./train_package/8/netfile
the portfolio value train No.8 is 0.8978206 log_mean is -0.0003992017, the training time is 57 seconds
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

INFO:root:the number of training examples is 3432, of test examples is 270
the number of training examples is 3432, of test examples is 270
INFO:root:the step is 0
the step is 0
INFO:root:the raw omega is [0.0574015  0.08569077 0.08569077 0.08569077 0.08569077 0.08569077
 0.08569077 0.08569077 0.08569077 0.08569077 0.08569077 0.08569077]
